# Data Import

In [2]:
import kagglehub
# Download latest version
path = kagglehub.dataset_download("dogcdt/synapse")
print("Path to dataset files:", path)

Using Colab cache for faster access to the 'synapse' dataset.
Path to dataset files: /kaggle/input/synapse


# Losses and Metrics

In [3]:
pip install medpy

In [4]:
import numpy as np
import torch
from medpy import metric
from scipy.ndimage import zoom
import torch.nn as nn
import SimpleITK as sitk

## metrics
def calculate_metric_percase(pred, gt):
    pred[pred > 0] = 1
    gt[gt > 0] = 1
    if pred.sum() > 0 and gt.sum()>0:
        dice = metric.binary.dc(pred, gt)
        hd95 = metric.binary.hd95(pred, gt)
        jac = metric.binary.jc(pred, gt)
        return dice, jac, hd95
    elif pred.sum() > 0 and gt.sum()==0:
        return 1, 1, 0
    else:
        return 0, 0, 0

def multilabel_metric(pred, gt, num_classes):
    gt = gt.squeeze(0)
    metric_list = []
    for i in range(1, num_classes):
        metric_list.append(calculate_metric_percase(pred == i, gt == i))
    return metric_list  # list, lenth=num_classes-1, 每个元素含有(x,y,z)

# losses
class BinaryDiceLoss(nn.Module):
    def init(self):
        super(BinaryDiceLoss, self).init()

    def forward(self, input, targets):
        # 获取每个批次的大小 N
        N = targets.size()[0]
        # 平滑变量
        smooth = 1
        # 将宽高 reshape 到同一纬度
        input_flat = input.view(N, -1)
        targets_flat = targets.view(N, -1)
        # 计算交集
        intersection = input_flat * targets_flat
        N_dice_eff = (2 * intersection.sum(1) + smooth) / (input_flat.sum(1) + targets_flat.sum(1) + smooth)
        # 计算一个批次中平均每张图的损失
        loss = 1 - N_dice_eff.sum() / N
        return loss


class MultiClassDiceLoss(nn.Module):
    def init(self, weight=None, ignore_index=None, **kwargs):
        super(MultiClassDiceLoss, self).init()
        self.weight = weight
        self.ignore_index = ignore_index
        self.kwargs = kwargs
    def forward(self, input, target):
        """
        input tesor of shape = (N, C, H, W)
        target tensor of shape = (N, H, W)
        """
        # 先将 target 进行 one-hot 处理，转换为 (N, C, H, W)
        # target[target==255.] = 0.
        nclass = input.shape[1]
        target = F.one_hot(target.long(), nclass)
        target = target.reshape(input.shape[0],input.shape[1],input.shape[2],-1)

        assert input.shape == target.shape, "predict & target shape do not match"
        binaryDiceLoss = BinaryDiceLoss()
        total_loss = 0
        # 归一化输出
        logits = F.softmax(input, dim=1)
        C = target.shape[1]
        # 遍历 channel，得到每个类别的二分类 DiceLoss
        for i in range(C):
            dice_loss = binaryDiceLoss(logits[:, i], target[:, i])
            total_loss += dice_loss
            # 每个类别的平均 dice_loss
        return total_loss / C

# DataSet implementation

In [5]:
import random
import h5py
import numpy as np
import torch
from scipy import ndimage
from scipy.ndimage import zoom
from torch.utils.data import Dataset
from pathlib import Path


# -------------------------
# Augmentations
# -------------------------
def random_rot_flip(image, label):
    k = np.random.randint(0, 4)
    image = np.rot90(image, k)
    label = np.rot90(label, k)
    axis = np.random.randint(0, 2)
    image = np.flip(image, axis=axis).copy()
    label = np.flip(label, axis=axis).copy()
    return image, label


def random_rotate(image, label):
    angle = np.random.randint(-20, 20)
    image = ndimage.rotate(image, angle, order=0, reshape=False)
    label = ndimage.rotate(label, angle, order=0, reshape=False)
    return image, label


class RandomGenerator(object):
    def __init__(self, output_size):
        self.output_size = output_size  # (H, W)

    def __call__(self, sample):
        image, label = sample["image"], sample["label"]

        if random.random() > 0.5:
            image, label = random_rot_flip(image, label)
        elif random.random() > 0.5:
            image, label = random_rotate(image, label)

        x, y = image.shape
        if (x != self.output_size[0]) or (y != self.output_size[1]):
            image = zoom(image, (self.output_size[0] / x, self.output_size[1] / y), order=3)
            label = zoom(label, (self.output_size[0] / x, self.output_size[1] / y), order=0)

        image = torch.from_numpy(image.astype(np.float32)).unsqueeze(0)  # (1, H, W)
        label = torch.from_numpy(label.astype(np.int64))                 # (H, W)
        return {"image": image, "label": label}


# -------------------------
# Dataset
# -------------------------
class Synapse_dataset(Dataset):
    """
    Your structure:

    /root/.cache/kagglehub/datasets/dogcdt/synapse/versions/1/Synapse/
        train_npz/
        test_vol_h5/   (or whatever your test folder is)
    /root/lists_synapse/
        train.txt
        test_vol.txt

    Train slices: <name>.npz with keys (image, label)
    Test volumes: <name>.npy.h5 OR <name>.h5 OR <name>.n5 with datasets (image, label)
    """

    def __init__(
        self,
        versions_root="/kaggle/input/synapse",
        split="train",
        transform=None,
        list_dir="/root/lists_synapse",
        data_subdir="Synapse",          # <-- IMPORTANT: your extra folder level
        train_folder="train_npz",
        test_folder="test_vol_h5",
        train_list="train.txt",
        test_list="test_vol.txt",
        verbose=True,
    ):
        self.transform = transform
        self.split = split.lower().strip()

        versions_root = Path(versions_root).expanduser().resolve()
        self.data_root = (versions_root / data_subdir).resolve()  # <-- /.../versions/1/Synapse

        list_dir = Path(list_dir).expanduser()
        self.list_dir = list_dir.resolve() if list_dir.is_absolute() else (self.data_root / list_dir).resolve()

        self.train_dir = (self.data_root / train_folder).resolve()
        self.test_dir = (self.data_root / test_folder).resolve()

        list_file = self.list_dir / (train_list if self.split == "train" else test_list)

        if verbose:
            print("VERSIONS ROOT:", versions_root)
            print("DATA ROOT    :", self.data_root)
            print("LIST DIR     :", self.list_dir)
            print("LIST FILE    :", list_file)
            print("TRAIN DIR    :", self.train_dir)
            print("TEST  DIR    :", self.test_dir)

        if not list_file.is_file():
            raise FileNotFoundError(f"Missing list file: {list_file}")

        self.sample_list = list_file.read_text().splitlines()

        if self.split == "train" and not self.train_dir.is_dir():
            raise FileNotFoundError(f"Missing folder: {self.train_dir}")
        if self.split != "train" and not self.test_dir.is_dir():
            raise FileNotFoundError(f"Missing folder: {self.test_dir}")

    def __len__(self):
        return len(self.sample_list)

    def _resolve_train_path(self, name: str) -> Path:
        return self.train_dir / f"{name}.npz"

    def _resolve_test_path(self, name: str) -> Path:
        candidates = [
            self.test_dir / f"{name}.npy.h5",
            self.test_dir / f"{name}.h5",
            self.test_dir / f"{name}.n5",
        ]
        for p in candidates:
            if p.is_file():
                return p
        return candidates[0]

    def show_paths(self, max_items=20):
        n = min(len(self.sample_list), max_items)
        print(f"Showing {n}/{len(self.sample_list)} paths for split='{self.split}'")
        for i in range(n):
            name = self.sample_list[i].strip()
            p = self._resolve_train_path(name) if self.split == "train" else self._resolve_test_path(name)
            print(p, "| exists =", p.exists())

    def __getitem__(self, idx):
        name = self.sample_list[idx].strip()

        if self.split == "train":
            data_path = self._resolve_train_path(name)
            if not data_path.is_file():
                raise FileNotFoundError(f"Missing train npz: {data_path}")

            data = np.load(str(data_path))
            image, label = data["image"], data["label"]

        else:
            h5_path = self._resolve_test_path(name)
            if not h5_path.is_file():
                raise FileNotFoundError(
                    "Missing test volume. Tried: "
                    f"{self.test_dir / (name + '.npy.h5')}, "
                    f"{self.test_dir / (name + '.h5')}, "
                    f"{self.test_dir / (name + '.n5')}"
                )

            with h5py.File(str(h5_path), "r") as data:
                image, label = data["image"][:], data["label"][:]

        sample = {"image": image, "label": label}
        if self.transform is not None:
            sample = self.transform(sample)

        sample["case_name"] = name
        return sample


# -------------------------
# Quick local test
# -------------------------
if __name__ == "__main__":
    versions_root = "/kaggle/input/synapse"
    list_dir = "/root/lists_synapse"

    ds_train = Synapse_dataset(versions_root=versions_root, split="train", transform=None, list_dir=list_dir)
    ds_train.show_paths(max_items=30)

    ds_test = Synapse_dataset(versions_root=versions_root, split="test", transform=None, list_dir=list_dir)
    ds_test.show_paths(max_items=10)

VERSIONS ROOT: /kaggle/input/synapse
DATA ROOT    : /kaggle/input/synapse/Synapse
LIST DIR     : /root/lists_synapse
LIST FILE    : /root/lists_synapse/train.txt
TRAIN DIR    : /kaggle/input/synapse/Synapse/train_npz
TEST  DIR    : /kaggle/input/synapse/Synapse/test_vol_h5
Showing 30/2211 paths for split='train'
/kaggle/input/synapse/Synapse/train_npz/case0031_slice000.npz | exists = True
/kaggle/input/synapse/Synapse/train_npz/case0031_slice001.npz | exists = True
/kaggle/input/synapse/Synapse/train_npz/case0031_slice002.npz | exists = True
/kaggle/input/synapse/Synapse/train_npz/case0031_slice003.npz | exists = True
/kaggle/input/synapse/Synapse/train_npz/case0031_slice004.npz | exists = True
/kaggle/input/synapse/Synapse/train_npz/case0031_slice005.npz | exists = True
/kaggle/input/synapse/Synapse/train_npz/case0031_slice006.npz | exists = True
/kaggle/input/synapse/Synapse/train_npz/case0031_slice007.npz | exists = True
/kaggle/input/synapse/Synapse/train_npz/case0031_slice008.npz 

# Unext Architecture

In [1]:
pip install -U openmim

In [6]:
pip install -U mmcv-lite

  Using cached addict-2.4.0-py3-none-any.whl.metadata (1.0 kB)
  Using cached mmengine-0.10.7-py3-none-any.whl.metadata (20 kB)
  Using cached yapf-0.43.0-py3-none-any.whl.metadata (46 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 732.3/732.3 kB 15.1 MB/s eta 0:00:00
Using cached mmengine-0.10.7-py3-none-any.whl (452 kB)
Using cached addict-2.4.0-py3-none-any.whl (3.8 kB)
Using cached yapf-0.43.0-py3-none-any.whl (256 kB)


## 2 version premier version avec mmcv et une deuxieme sans

In [7]:

import torch
from torch import nn
import torch
from torch import nn

import torch.nn.functional as F

#from utils import *
__all__ = ['UNext']

import timm
from timm.models.layers import DropPath, to_2tuple, trunc_normal_
import types
import math
from abc import ABCMeta, abstractmethod
from mmcv.cnn import ConvModule
import pdb



def conv1x1(in_planes: int, out_planes: int, stride: int = 1) -> nn.Conv2d:
    """1x1 convolution"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=1, bias=False)


# def shift(dim):
#     x_shift = [ torch.roll(x_c, shift, dim) for x_c, shift in zip(xs, range(-self.pad, self.pad +1))]
#     x_cat = torch.cat(x_shift, 1)
#     x_cat = torch.narrow(x_cat, 2, self.pad, H)
#     x_cat = torch.narrow(x_cat, 3, self.pad, W)
#     return x_cat

class shiftmlp(nn.Module):
    def __init__(self, in_features, hidden_features=None, out_features=None, act_layer=nn.GELU, drop=0., shift_size=5):
        super().__init__()
        out_features = out_features or in_features
        hidden_features = hidden_features or in_features
        self.dim = in_features
        self.fc1 = nn.Linear(in_features, hidden_features)
        self.dwconv = DWConv(hidden_features)
        self.act = act_layer()
        self.fc2 = nn.Linear(hidden_features, out_features)
        self.drop = nn.Dropout(drop)

        self.shift_size = shift_size
        self.pad = shift_size // 2


        self.apply(self._init_weights)

    def _init_weights(self, m):
        if isinstance(m, nn.Linear):
            trunc_normal_(m.weight, std=.02)
            if isinstance(m, nn.Linear) and m.bias is not None:
                nn.init.constant_(m.bias, 0)
        elif isinstance(m, nn.LayerNorm):
            nn.init.constant_(m.bias, 0)
            nn.init.constant_(m.weight, 1.0)
        elif isinstance(m, nn.Conv2d):
            fan_out = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
            fan_out //= m.groups
            m.weight.data.normal_(0, math.sqrt(2.0 / fan_out))
            if m.bias is not None:
                m.bias.data.zero_()

    #     def shift(x, dim):
    #         x = F.pad(x, "constant", 0)
    #         x = torch.chunk(x, shift_size, 1)
    #         x = [ torch.roll(x_c, shift, dim) for x_s, shift in zip(x, range(-pad, pad+1))]
    #         x = torch.cat(x, 1)
    #         return x[:, :, pad:-pad, pad:-pad]

    def forward(self, x, H, W):
        # pdb.set_trace()
        B, N, C = x.shape

        xn = x.transpose(1, 2).view(B, C, H, W).contiguous()
        xn = F.pad(xn, (self.pad, self.pad, self.pad, self.pad) , "constant", 0)
        xs = torch.chunk(xn, self.shift_size, 1)
        x_shift = [torch.roll(x_c, shift, 2) for x_c, shift in zip(xs, range(-self.pad, self.pad +1))]
        x_cat = torch.cat(x_shift, 1)
        x_cat = torch.narrow(x_cat, 2, self.pad, H)
        x_s = torch.narrow(x_cat, 3, self.pad, W)


        x_s = x_s.reshape(B ,C , H *W).contiguous()
        x_shift_r = x_s.transpose(1 ,2)


        x = self.fc1(x_shift_r)

        x = self.dwconv(x, H, W)
        x = self.act(x)
        x = self.drop(x)

        xn = x.transpose(1, 2).view(B, C, H, W).contiguous()
        xn = F.pad(xn, (self.pad, self.pad, self.pad, self.pad) , "constant", 0)
        xs = torch.chunk(xn, self.shift_size, 1)
        x_shift = [torch.roll(x_c, shift, 3) for x_c, shift in zip(xs, range(-self.pad, self.pad +1))]
        x_cat = torch.cat(x_shift, 1)
        x_cat = torch.narrow(x_cat, 2, self.pad, H)
        x_s = torch.narrow(x_cat, 3, self.pad, W)
        x_s = x_s.reshape(B ,C , H *W).contiguous()
        x_shift_c = x_s.transpose(1 ,2)

        x = self.fc2(x_shift_c)
        x = self.drop(x)
        return x



class shiftedBlock(nn.Module):
    def __init__(self, dim, num_heads, mlp_ratio=4., qkv_bias=False, qk_scale=None, drop=0., attn_drop=0.,
                 drop_path=0., act_layer=nn.GELU, norm_layer=nn.LayerNorm, sr_ratio=1):
        super().__init__()


        self.drop_path = DropPath(drop_path) if drop_path > 0. else nn.Identity()
        self.norm2 = norm_layer(dim)
        mlp_hidden_dim = int(dim * mlp_ratio)
        self.mlp = shiftmlp(in_features=dim, hidden_features=mlp_hidden_dim, act_layer=act_layer, drop=drop)
        self.apply(self._init_weights)

    def _init_weights(self, m):
        if isinstance(m, nn.Linear):
            trunc_normal_(m.weight, std=.02)
            if isinstance(m, nn.Linear) and m.bias is not None:
                nn.init.constant_(m.bias, 0)
        elif isinstance(m, nn.LayerNorm):
            nn.init.constant_(m.bias, 0)
            nn.init.constant_(m.weight, 1.0)
        elif isinstance(m, nn.Conv2d):
            fan_out = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
            fan_out //= m.groups
            m.weight.data.normal_(0, math.sqrt(2.0 / fan_out))
            if m.bias is not None:
                m.bias.data.zero_()

    def forward(self, x, H, W):

        x = x + self.drop_path(self.mlp(self.norm2(x), H, W))
        return x


class DWConv(nn.Module):
    def __init__(self, dim=768):
        super(DWConv, self).__init__()
        self.dwconv = nn.Conv2d(dim, dim, 3, 1, 1, bias=True, groups=dim)

    def forward(self, x, H, W):
        B, N, C = x.shape
        x = x.transpose(1, 2).view(B, C, H, W)
        x = self.dwconv(x)
        x = x.flatten(2).transpose(1, 2)

        return x

class OverlapPatchEmbed(nn.Module):
    """ Image to Patch Embedding
    """

    def __init__(self, img_size=224, patch_size=7, stride=4, in_chans=3, embed_dim=768):
        super().__init__()
        img_size = to_2tuple(img_size)
        patch_size = to_2tuple(patch_size)

        self.img_size = img_size
        self.patch_size = patch_size
        self.H, self.W = img_size[0] // patch_size[0], img_size[1] // patch_size[1]
        self.num_patches = self.H * self.W
        self.proj = nn.Conv2d(in_chans, embed_dim, kernel_size=patch_size, stride=stride,
                              padding=(patch_size[0] // 2, patch_size[1] // 2))
        self.norm = nn.LayerNorm(embed_dim)

        self.apply(self._init_weights)

    def _init_weights(self, m):
        if isinstance(m, nn.Linear):
            trunc_normal_(m.weight, std=.02)
            if isinstance(m, nn.Linear) and m.bias is not None:
                nn.init.constant_(m.bias, 0)
        elif isinstance(m, nn.LayerNorm):
            nn.init.constant_(m.bias, 0)
            nn.init.constant_(m.weight, 1.0)
        elif isinstance(m, nn.Conv2d):
            fan_out = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
            fan_out //= m.groups
            m.weight.data.normal_(0, math.sqrt(2.0 / fan_out))
            if m.bias is not None:
                m.bias.data.zero_()

    def forward(self, x):
        x = self.proj(x)
        _, _, H, W = x.shape
        x = x.flatten(2).transpose(1, 2)
        x = self.norm(x)

        return x, H, W


class UNext(nn.Module):

    ## Conv 3 + MLP 2 + shifted MLP

    def __init__(self,  num_classes, input_channels=3, deep_supervision=False ,img_size=224, patch_size=16, in_chans=3,  embed_dims=[ 128, 160, 256],
                 num_heads=[1, 2, 4, 8], mlp_ratios=[4, 4, 4, 4], qkv_bias=False, qk_scale=None, drop_rate=0.,
                 attn_drop_rate=0., drop_path_rate=0., norm_layer=nn.LayerNorm,
                 depths=[1, 1, 1], sr_ratios=[8, 4, 2, 1], **kwargs):
        super().__init__()

        self.encoder1 = nn.Conv2d(3, 16, 3, stride=1, padding=1)
        self.encoder2 = nn.Conv2d(16, 32, 3, stride=1, padding=1)
        self.encoder3 = nn.Conv2d(32, 128, 3, stride=1, padding=1)

        self.ebn1 = nn.BatchNorm2d(16)
        self.ebn2 = nn.BatchNorm2d(32)
        self.ebn3 = nn.BatchNorm2d(128)

        self.norm3 = norm_layer(embed_dims[1])
        self.norm4 = norm_layer(embed_dims[2])

        self.dnorm3 = norm_layer(160)
        self.dnorm4 = norm_layer(128)

        dpr = [x.item() for x in torch.linspace(0, drop_path_rate, sum(depths))]

        self.block1 = nn.ModuleList([shiftedBlock(
            dim=embed_dims[1], num_heads=num_heads[0], mlp_ratio=1, qkv_bias=qkv_bias, qk_scale=qk_scale,
            drop=drop_rate, attn_drop=attn_drop_rate, drop_path=dpr[0], norm_layer=norm_layer,
            sr_ratio=sr_ratios[0])])

        self.block2 = nn.ModuleList([shiftedBlock(
            dim=embed_dims[2], num_heads=num_heads[0], mlp_ratio=1, qkv_bias=qkv_bias, qk_scale=qk_scale,
            drop=drop_rate, attn_drop=attn_drop_rate, drop_path=dpr[1], norm_layer=norm_layer,
            sr_ratio=sr_ratios[0])])

        self.dblock1 = nn.ModuleList([shiftedBlock(
            dim=embed_dims[1], num_heads=num_heads[0], mlp_ratio=1, qkv_bias=qkv_bias, qk_scale=qk_scale,
            drop=drop_rate, attn_drop=attn_drop_rate, drop_path=dpr[0], norm_layer=norm_layer,
            sr_ratio=sr_ratios[0])])

        self.dblock2 = nn.ModuleList([shiftedBlock(
            dim=embed_dims[0], num_heads=num_heads[0], mlp_ratio=1, qkv_bias=qkv_bias, qk_scale=qk_scale,
            drop=drop_rate, attn_drop=attn_drop_rate, drop_path=dpr[1], norm_layer=norm_layer,
            sr_ratio=sr_ratios[0])])

        self.patch_embed3 = OverlapPatchEmbed(img_size=img_size // 4, patch_size=3, stride=2, in_chans=embed_dims[0],
                                              embed_dim=embed_dims[1])
        self.patch_embed4 = OverlapPatchEmbed(img_size=img_size // 8, patch_size=3, stride=2, in_chans=embed_dims[1],
                                              embed_dim=embed_dims[2])

        self.decoder1 = nn.Conv2d(256, 160, 3, stride=1 ,padding=1)
        self.decoder2 =   nn.Conv2d(160, 128, 3, stride=1, padding=1)
        self.decoder3 =   nn.Conv2d(128, 32, 3, stride=1, padding=1)
        self.decoder4 =   nn.Conv2d(32, 16, 3, stride=1, padding=1)
        self.decoder5 =   nn.Conv2d(16, 16, 3, stride=1, padding=1)

        self.dbn1 = nn.BatchNorm2d(160)
        self.dbn2 = nn.BatchNorm2d(128)
        self.dbn3 = nn.BatchNorm2d(32)
        self.dbn4 = nn.BatchNorm2d(16)

        self.final = nn.Conv2d(16, num_classes, kernel_size=1)

        self.soft = nn.Softmax(dim =1)

    def forward(self, x):

        B = x.shape[0]
        ### Encoder
        ### Conv Stage

        ### Stage 1
        out = F.relu(F.max_pool2d(self.ebn1(self.encoder1(x)) ,2 ,2))
        t1 = out
        ### Stage 2
        out = F.relu(F.max_pool2d(self.ebn2(self.encoder2(out)) ,2 ,2))
        t2 = out
        ### Stage 3
        out = F.relu(F.max_pool2d(self.ebn3(self.encoder3(out)) ,2 ,2))
        t3 = out

        ### Tokenized MLP Stage
        ### Stage 4

        out ,H ,W = self.patch_embed3(out)
        for i, blk in enumerate(self.block1):
            out = blk(out, H, W)
        out = self.norm3(out)
        out = out.reshape(B, H, W, -1).permute(0, 3, 1, 2).contiguous()
        t4 = out

        ### Bottleneck

        out ,H, W = self.patch_embed4(out)
        for i, blk in enumerate(self.block2):
            out = blk(out, H, W)
        out = self.norm4(out)
        out = out.reshape(B, H, W, -1).permute(0, 3, 1, 2).contiguous()

        ### Stage 4

        out = F.relu(F.interpolate(self.dbn1(self.decoder1(out)), scale_factor=(2, 2), mode='bilinear'))

        out = torch.add(out, t4)
        _, _, H, W = out.shape
        out = out.flatten(2).transpose(1, 2)
        for i, blk in enumerate(self.dblock1):
            out = blk(out, H, W)

        ### Stage 3

        out = self.dnorm3(out)
        out = out.reshape(B, H, W, -1).permute(0, 3, 1, 2).contiguous()
        out = F.relu(F.interpolate(self.dbn2(self.decoder2(out)), scale_factor=(2, 2), mode='bilinear'))
        out = torch.add(out, t3)
        _, _, H, W = out.shape
        out = out.flatten(2).transpose(1, 2)

        for i, blk in enumerate(self.dblock2):
            out = blk(out, H, W)

        out = self.dnorm4(out)
        out = out.reshape(B, H, W, -1).permute(0, 3, 1, 2).contiguous()

        out = F.relu(F.interpolate(self.dbn3(self.decoder3(out)), scale_factor=(2, 2), mode='bilinear'))
        out = torch.add(out, t2)
        out = F.relu(F.interpolate(self.dbn4(self.decoder4(out)), scale_factor=(2, 2), mode='bilinear'))
        out = torch.add(out, t1)
        out = F.relu(F.interpolate(self.decoder5(out), scale_factor=(2, 2), mode='bilinear'))

        # out = self.final(out)
        # out = F.sigmoid(self.final(out).squeeze(1))
        out = self.final(out)

        return out


class UNext_S(nn.Module):

    ## Conv 3 + MLP 2 + shifted MLP w less parameters

    def __init__(self, num_classes, input_channels=3, deep_supervision=False, img_size=224, patch_size=16, in_chans=3,
                 embed_dims=[32, 64, 128, 512],
                 num_heads=[1, 2, 4, 8], mlp_ratios=[4, 4, 4, 4], qkv_bias=False, qk_scale=None, drop_rate=0.,
                 attn_drop_rate=0., drop_path_rate=0., norm_layer=nn.LayerNorm,
                 depths=[1, 1, 1], sr_ratios=[8, 4, 2, 1], **kwargs):
        super().__init__()

        self.encoder1 = nn.Conv2d(3, 8, 3, stride=1, padding=1)
        self.encoder2 = nn.Conv2d(8, 16, 3, stride=1, padding=1)
        self.encoder3 = nn.Conv2d(16, 32, 3, stride=1, padding=1)

        self.ebn1 = nn.BatchNorm2d(8)
        self.ebn2 = nn.BatchNorm2d(16)
        self.ebn3 = nn.BatchNorm2d(32)

        self.norm3 = norm_layer(embed_dims[1])
        self.norm4 = norm_layer(embed_dims[2])

        self.dnorm3 = norm_layer(64)
        self.dnorm4 = norm_layer(32)

        dpr = [x.item() for x in torch.linspace(0, drop_path_rate, sum(depths))]

        self.block1 = nn.ModuleList([shiftedBlock(
            dim=embed_dims[1], num_heads=num_heads[0], mlp_ratio=1, qkv_bias=qkv_bias, qk_scale=qk_scale,
            drop=drop_rate, attn_drop=attn_drop_rate, drop_path=dpr[0], norm_layer=norm_layer,
            sr_ratio=sr_ratios[0])])

        self.block2 = nn.ModuleList([shiftedBlock(
            dim=embed_dims[2], num_heads=num_heads[0], mlp_ratio=1, qkv_bias=qkv_bias, qk_scale=qk_scale,
            drop=drop_rate, attn_drop=attn_drop_rate, drop_path=dpr[1], norm_layer=norm_layer,
            sr_ratio=sr_ratios[0])])

        self.dblock1 = nn.ModuleList([shiftedBlock(
            dim=embed_dims[1], num_heads=num_heads[0], mlp_ratio=1, qkv_bias=qkv_bias, qk_scale=qk_scale,
            drop=drop_rate, attn_drop=attn_drop_rate, drop_path=dpr[0], norm_layer=norm_layer,
            sr_ratio=sr_ratios[0])])

        self.dblock2 = nn.ModuleList([shiftedBlock(
            dim=embed_dims[0], num_heads=num_heads[0], mlp_ratio=1, qkv_bias=qkv_bias, qk_scale=qk_scale,
            drop=drop_rate, attn_drop=attn_drop_rate, drop_path=dpr[1], norm_layer=norm_layer,
            sr_ratio=sr_ratios[0])])

        self.patch_embed3 = OverlapPatchEmbed(img_size=img_size // 4, patch_size=3, stride=2, in_chans=embed_dims[0],
                                              embed_dim=embed_dims[1])
        self.patch_embed4 = OverlapPatchEmbed(img_size=img_size // 8, patch_size=3, stride=2, in_chans=embed_dims[1],
                                              embed_dim=embed_dims[2])

        self.decoder1 = nn.Conv2d(128, 64, 3, stride=1, padding=1)
        self.decoder2 = nn.Conv2d(64, 32, 3, stride=1, padding=1)
        self.decoder3 = nn.Conv2d(32, 16, 3, stride=1, padding=1)
        self.decoder4 = nn.Conv2d(16, 8, 3, stride=1, padding=1)
        self.decoder5 = nn.Conv2d(8, 8, 3, stride=1, padding=1)

        self.dbn1 = nn.BatchNorm2d(64)
        self.dbn2 = nn.BatchNorm2d(32)
        self.dbn3 = nn.BatchNorm2d(16)
        self.dbn4 = nn.BatchNorm2d(8)

        self.final = nn.Conv2d(8, num_classes, kernel_size=1)

        self.soft = nn.Softmax(dim=1)

    def forward(self, x):

        B = x.shape[0]
        ### Encoder
        ### Conv Stage

        ### Stage 1
        out = F.relu(F.max_pool2d(self.ebn1(self.encoder1(x)), 2, 2))
        t1 = out
        ### Stage 2
        out = F.relu(F.max_pool2d(self.ebn2(self.encoder2(out)), 2, 2))
        t2 = out
        ### Stage 3
        out = F.relu(F.max_pool2d(self.ebn3(self.encoder3(out)), 2, 2))
        t3 = out

        ### Tokenized MLP Stage
        ### Stage 4

        out, H, W = self.patch_embed3(out)
        for i, blk in enumerate(self.block1):
            out = blk(out, H, W)
        out = self.norm3(out)
        out = out.reshape(B, H, W, -1).permute(0, 3, 1, 2).contiguous()
        t4 = out

        ### Bottleneck

        out, H, W = self.patch_embed4(out)
        for i, blk in enumerate(self.block2):
            out = blk(out, H, W)
        out = self.norm4(out)
        out = out.reshape(B, H, W, -1).permute(0, 3, 1, 2).contiguous()

        ### Stage 4

        out = F.relu(F.interpolate(self.dbn1(self.decoder1(out)), scale_factor=(2, 2), mode='bilinear'))

        out = torch.add(out, t4)
        _, _, H, W = out.shape
        out = out.flatten(2).transpose(1, 2)
        for i, blk in enumerate(self.dblock1):
            out = blk(out, H, W)

        ### Stage 3

        out = self.dnorm3(out)
        out = out.reshape(B, H, W, -1).permute(0, 3, 1, 2).contiguous()
        out = F.relu(F.interpolate(self.dbn2(self.decoder2(out)), scale_factor=(2, 2), mode='bilinear'))
        out = torch.add(out, t3)
        _, _, H, W = out.shape
        out = out.flatten(2).transpose(1, 2)

        for i, blk in enumerate(self.dblock2):
            out = blk(out, H, W)

        out = self.dnorm4(out)
        out = out.reshape(B, H, W, -1).permute(0, 3, 1, 2).contiguous()

        out = F.relu(F.interpolate(self.dbn3(self.decoder3(out)), scale_factor=(2, 2), mode='bilinear'))
        out = torch.add(out, t2)
        out = F.relu(F.interpolate(self.dbn4(self.decoder4(out)), scale_factor=(2, 2), mode='bilinear'))
        out = torch.add(out, t1)
        out = F.relu(F.interpolate(self.decoder5(out), scale_factor=(2, 2), mode='bilinear'))

        return self.final(out)

# EOF
model = UNext(num_classes=1, input_channels=3, deep_supervision=True).cuda()
inp = torch.rand((2, 3, 224, 224)).cuda()
out = model(inp)
print(out.shape)

/usr/local/lib/python3.12/dist-packages/timm/models/layers/__init__.py:49: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


torch.Size([2, 1, 224, 224])


In [9]:
__all__ = ['UNext', 'UNext_S']

import timm
from timm.models.layers import DropPath, to_2tuple, trunc_normal_
import math


class DWConv(nn.Module):
    def __init__(self, dim=768):
        super().__init__()
        self.dwconv = nn.Conv2d(dim, dim, 3, 1, 1, bias=True, groups=dim)

    def forward(self, x, H, W):
        B, N, C = x.shape
        x = x.transpose(1, 2).view(B, C, H, W)
        x = self.dwconv(x)
        x = x.flatten(2).transpose(1, 2)
        return x


class shiftmlp(nn.Module):
    def __init__(self, in_features, hidden_features=None, out_features=None,
                 act_layer=nn.GELU, drop=0., shift_size=5):
        super().__init__()
        out_features = out_features or in_features
        hidden_features = hidden_features or in_features

        self.dim = in_features
        self.fc1 = nn.Linear(in_features, hidden_features)
        self.dwconv = DWConv(hidden_features)
        self.act = act_layer()
        self.fc2 = nn.Linear(hidden_features, out_features)
        self.drop = nn.Dropout(drop)

        self.shift_size = shift_size
        self.pad = shift_size // 2
        self.apply(self._init_weights)

    def _init_weights(self, m):
        if isinstance(m, nn.Linear):
            trunc_normal_(m.weight, std=.02)
            if m.bias is not None:
                nn.init.constant_(m.bias, 0)
        elif isinstance(m, nn.LayerNorm):
            nn.init.constant_(m.bias, 0)
            nn.init.constant_(m.weight, 1.0)
        elif isinstance(m, nn.Conv2d):
            fan_out = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
            fan_out //= m.groups
            m.weight.data.normal_(0, math.sqrt(2.0 / fan_out))
            if m.bias is not None:
                m.bias.data.zero_()

    def forward(self, x, H, W):
        B, N, C = x.shape

        xn = x.transpose(1, 2).view(B, C, H, W).contiguous()
        xn = F.pad(xn, (self.pad, self.pad, self.pad, self.pad), "constant", 0)
        xs = torch.chunk(xn, self.shift_size, 1)
        x_shift = [torch.roll(x_c, shift, 2) for x_c, shift in zip(xs, range(-self.pad, self.pad + 1))]
        x_cat = torch.cat(x_shift, 1)
        x_cat = torch.narrow(x_cat, 2, self.pad, H)
        x_s = torch.narrow(x_cat, 3, self.pad, W)
        x_s = x_s.reshape(B, C, H * W).contiguous()
        x_shift_r = x_s.transpose(1, 2)

        x = self.fc1(x_shift_r)
        x = self.dwconv(x, H, W)
        x = self.act(x)
        x = self.drop(x)

        xn = x.transpose(1, 2).view(B, C, H, W).contiguous()
        xn = F.pad(xn, (self.pad, self.pad, self.pad, self.pad), "constant", 0)
        xs = torch.chunk(xn, self.shift_size, 1)
        x_shift = [torch.roll(x_c, shift, 3) for x_c, shift in zip(xs, range(-self.pad, self.pad + 1))]
        x_cat = torch.cat(x_shift, 1)
        x_cat = torch.narrow(x_cat, 2, self.pad, H)
        x_s = torch.narrow(x_cat, 3, self.pad, W)
        x_s = x_s.reshape(B, C, H * W).contiguous()
        x_shift_c = x_s.transpose(1, 2)

        x = self.fc2(x_shift_c)
        x = self.drop(x)
        return x


class shiftedBlock(nn.Module):
    def __init__(self, dim, num_heads, mlp_ratio=4., qkv_bias=False,
                 qk_scale=None, drop=0., attn_drop=0., drop_path=0.,
                 act_layer=nn.GELU, norm_layer=nn.LayerNorm, sr_ratio=1):
        super().__init__()
        self.drop_path = DropPath(drop_path) if drop_path > 0. else nn.Identity()
        self.norm2 = norm_layer(dim)
        mlp_hidden_dim = int(dim * mlp_ratio)
        self.mlp = shiftmlp(in_features=dim, hidden_features=mlp_hidden_dim,
                            act_layer=act_layer, drop=drop)
        self.apply(self._init_weights)

    def _init_weights(self, m):
        if isinstance(m, nn.Linear):
            trunc_normal_(m.weight, std=.02)
            if m.bias is not None:
                nn.init.constant_(m.bias, 0)
        elif isinstance(m, nn.LayerNorm):
            nn.init.constant_(m.bias, 0)
            nn.init.constant_(m.weight, 1.0)
        elif isinstance(m, nn.Conv2d):
            fan_out = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
            fan_out //= m.groups
            m.weight.data.normal_(0, math.sqrt(2.0 / fan_out))
            if m.bias is not None:
                m.bias.data.zero_()

    def forward(self, x, H, W):
        x = x + self.drop_path(self.mlp(self.norm2(x), H, W))
        return x


class OverlapPatchEmbed(nn.Module):
    def __init__(self, img_size=224, patch_size=7, stride=4, in_chans=3, embed_dim=768):
        super().__init__()
        img_size = to_2tuple(img_size)
        patch_size = to_2tuple(patch_size)
        self.img_size = img_size
        self.patch_size = patch_size
        self.H, self.W = img_size[0] // patch_size[0], img_size[1] // patch_size[1]
        self.num_patches = self.H * self.W

        self.proj = nn.Conv2d(in_chans, embed_dim, kernel_size=patch_size,
                              stride=stride, padding=(patch_size[0] // 2, patch_size[1] // 2))
        self.norm = nn.LayerNorm(embed_dim)
        self.apply(self._init_weights)

    def _init_weights(self, m):
        if isinstance(m, nn.Linear):
            trunc_normal_(m.weight, std=.02)
            if m.bias is not None:
                nn.init.constant_(m.bias, 0)
        elif isinstance(m, nn.LayerNorm):
            nn.init.constant_(m.bias, 0)
            nn.init.constant_(m.weight, 1.0)
        elif isinstance(m, nn.Conv2d):
            fan_out = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
            fan_out //= m.groups
            m.weight.data.normal_(0, math.sqrt(2.0 / fan_out))
            if m.bias is not None:
                m.bias.data.zero_()

    def forward(self, x):
        x = self.proj(x)
        _, _, H, W = x.shape
        x = x.flatten(2).transpose(1, 2)
        x = self.norm(x)
        return x, H, W


class UNext(nn.Module):
    def __init__(self, num_classes, input_channels=3, deep_supervision=False,
                 img_size=224, patch_size=16, in_chans=3,
                 embed_dims=[128, 160, 256],
                 num_heads=[1, 2, 4, 8],
                 mlp_ratios=[4, 4, 4, 4],
                 qkv_bias=False, qk_scale=None, drop_rate=0.,
                 attn_drop_rate=0., drop_path_rate=0.,
                 norm_layer=nn.LayerNorm, depths=[1, 1, 1],
                 sr_ratios=[8, 4, 2, 1], **kwargs):
        super().__init__()

        self.encoder1 = nn.Conv2d(3, 16, 3, stride=1, padding=1)
        self.encoder2 = nn.Conv2d(16, 32, 3, stride=1, padding=1)
        self.encoder3 = nn.Conv2d(32, 128, 3, stride=1, padding=1)

        self.ebn1 = nn.BatchNorm2d(16)
        self.ebn2 = nn.BatchNorm2d(32)
        self.ebn3 = nn.BatchNorm2d(128)

        self.norm3 = norm_layer(embed_dims[1])
        self.norm4 = norm_layer(embed_dims[2])

        self.dnorm3 = norm_layer(160)
        self.dnorm4 = norm_layer(128)

        dpr = [x.item() for x in torch.linspace(0, drop_path_rate, sum(depths))]

        self.block1 = nn.ModuleList([shiftedBlock(
            dim=embed_dims[1], num_heads=num_heads[0], mlp_ratio=1, qkv_bias=qkv_bias,
            qk_scale=qk_scale, drop=drop_rate, attn_drop=attn_drop_rate,
            drop_path=dpr[0], norm_layer=norm_layer, sr_ratio=sr_ratios[0])])

        self.block2 = nn.ModuleList([shiftedBlock(
            dim=embed_dims[2], num_heads=num_heads[0], mlp_ratio=1, qkv_bias=qkv_bias,
            qk_scale=qk_scale, drop=drop_rate, attn_drop=attn_drop_rate,
            drop_path=dpr[1], norm_layer=norm_layer, sr_ratio=sr_ratios[0])])

        self.dblock1 = nn.ModuleList([shiftedBlock(
            dim=embed_dims[1], num_heads=num_heads[0], mlp_ratio=1, qkv_bias=qkv_bias,
            qk_scale=qk_scale, drop=drop_rate, attn_drop=attn_drop_rate,
            drop_path=dpr[0], norm_layer=norm_layer, sr_ratio=sr_ratios[0])])

        self.dblock2 = nn.ModuleList([shiftedBlock(
            dim=embed_dims[0], num_heads=num_heads[0], mlp_ratio=1, qkv_bias=qkv_bias,
            qk_scale=qk_scale, drop=drop_rate, attn_drop=attn_drop_rate,
            drop_path=dpr[1], norm_layer=norm_layer, sr_ratio=sr_ratios[0])])

        self.patch_embed3 = OverlapPatchEmbed(img_size=img_size // 4, patch_size=3, stride=2,
                                              in_chans=embed_dims[0], embed_dim=embed_dims[1])
        self.patch_embed4 = OverlapPatchEmbed(img_size=img_size // 8, patch_size=3, stride=2,
                                              in_chans=embed_dims[1], embed_dim=embed_dims[2])

        self.decoder1 = nn.Conv2d(256, 160, 3, stride=1, padding=1)
        self.decoder2 = nn.Conv2d(160, 128, 3, stride=1, padding=1)
        self.decoder3 = nn.Conv2d(128, 32, 3, stride=1, padding=1)
        self.decoder4 = nn.Conv2d(32, 16, 3, stride=1, padding=1)
        self.decoder5 = nn.Conv2d(16, 16, 3, stride=1, padding=1)

        self.dbn1 = nn.BatchNorm2d(160)
        self.dbn2 = nn.BatchNorm2d(128)
        self.dbn3 = nn.BatchNorm2d(32)
        self.dbn4 = nn.BatchNorm2d(16)

        self.final = nn.Conv2d(16, num_classes, kernel_size=1)

    def forward(self, x):
        B = x.shape[0]

        out = F.relu(F.max_pool2d(self.ebn1(self.encoder1(x)), 2, 2))
        t1 = out
        out = F.relu(F.max_pool2d(self.ebn2(self.encoder2(out)), 2, 2))
        t2 = out
        out = F.relu(F.max_pool2d(self.ebn3(self.encoder3(out)), 2, 2))
        t3 = out

        out, H, W = self.patch_embed3(out)
        for blk in self.block1:
            out = blk(out, H, W)
        out = self.norm3(out)
        out = out.reshape(B, H, W, -1).permute(0, 3, 1, 2).contiguous()
        t4 = out

        out, H, W = self.patch_embed4(out)
        for blk in self.block2:
            out = blk(out, H, W)
        out = self.norm4(out)
        out = out.reshape(B, H, W, -1).permute(0, 3, 1, 2).contiguous()

        out = F.relu(F.interpolate(self.dbn1(self.decoder1(out)), scale_factor=(2, 2), mode='bilinear'))
        out = torch.add(out, t4)
        _, _, H, W = out.shape
        out = out.flatten(2).transpose(1, 2)
        for blk in self.dblock1:
            out = blk(out, H, W)

        out = self.dnorm3(out)
        out = out.reshape(B, H, W, -1).permute(0, 3, 1, 2).contiguous()
        out = F.relu(F.interpolate(self.dbn2(self.decoder2(out)), scale_factor=(2, 2), mode='bilinear'))
        out = torch.add(out, t3)
        _, _, H, W = out.shape
        out = out.flatten(2).transpose(1, 2)
        for blk in self.dblock2:
            out = blk(out, H, W)

        out = self.dnorm4(out)
        out = out.reshape(B, H, W, -1).permute(0, 3, 1, 2).contiguous()

        out = F.relu(F.interpolate(self.dbn3(self.decoder3(out)), scale_factor=(2, 2), mode='bilinear'))
        out = torch.add(out, t2)
        out = F.relu(F.interpolate(self.dbn4(self.decoder4(out)), scale_factor=(2, 2), mode='bilinear'))
        out = torch.add(out, t1)
        out = F.relu(F.interpolate(self.decoder5(out), scale_factor=(2, 2), mode='bilinear'))

        out = self.final(out)
        return out

# Main

In [ ]:
import os
import sys
import random
import argparse
import datetime
import numpy as np
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset

from medpy import metric as medpy_metric
from google.colab import drive


# ============================================================
#                 Metrics (MedPy)
# ============================================================

def calculate_metric_percase(pred, gt):
    pred = pred.astype(np.uint8)
    gt = gt.astype(np.uint8)
    pred[pred > 0] = 1
    gt[gt > 0] = 1

    if gt.sum() == 0:
        return np.nan, np.nan, np.nan

    if pred.sum() == 0:
        return 0.0, 0.0, np.nan

    dice = medpy_metric.binary.dc(pred, gt)
    jac = medpy_metric.binary.jc(pred, gt)
    hd95 = medpy_metric.binary.hd95(pred, gt)
    return float(dice), float(jac), float(hd95)


def multilabel_metric(pred_2d, gt_2d, num_classes):
    metric_list = []
    for i in range(1, num_classes):
        metric_list.append(calculate_metric_percase(pred_2d == i, gt_2d == i))
    return metric_list


def init_running_metrics(num_classes: int):
    return {
        "dice_sum": {c: 0.0 for c in range(1, num_classes)},
        "jac_sum":  {c: 0.0 for c in range(1, num_classes)},
        "hd95_sum": {c: 0.0 for c in range(1, num_classes)},
        "count":    {c: 0   for c in range(1, num_classes)},
    }


def update_running_metrics(running, pred_2d, gt_2d, num_classes: int):
    mlist = multilabel_metric(pred_2d, gt_2d, num_classes)
    for cls_idx, (dice, jac, hd95) in enumerate(mlist, start=1):
        if not np.isnan(dice):
            running["dice_sum"][cls_idx] += float(dice)
        if not np.isnan(jac):
            running["jac_sum"][cls_idx] += float(jac)
        if not np.isnan(hd95):
            running["hd95_sum"][cls_idx] += float(hd95)
        if (not np.isnan(dice)) or (not np.isnan(jac)) or (not np.isnan(hd95)):
            running["count"][cls_idx] += 1
    return running


def finalize_metrics(running, num_classes: int):
    per_class = {}
    dice_vals, jac_vals, hd95_vals = [], [], []

    for c in range(1, num_classes):
        cnt = running["count"][c]
        if cnt > 0:
            d = running["dice_sum"][c] / cnt
            j = running["jac_sum"][c] / cnt
            h = running["hd95_sum"][c] / cnt
        else:
            d, j, h = np.nan, np.nan, np.nan

        per_class[c] = (float(d) if not np.isnan(d) else np.nan,
                        float(j) if not np.isnan(j) else np.nan,
                        float(h) if not np.isnan(h) else np.nan)

        dice_vals.append(d)
        jac_vals.append(j)
        hd95_vals.append(h)

    macro_dice = float(np.nanmean(dice_vals)) if len(dice_vals) else np.nan
    macro_jac  = float(np.nanmean(jac_vals))  if len(jac_vals)  else np.nan
    macro_hd95 = float(np.nanmean(hd95_vals)) if len(hd95_vals) else np.nan

    return per_class, (macro_dice, macro_jac, macro_hd95)


# ============================================================
#                 Losses (Dice excludes background)
# ============================================================

class BinaryDiceLoss(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, input, targets):
        N = targets.size(0)
        smooth = 1.0
        input_flat = input.view(N, -1)
        targets_flat = targets.view(N, -1)
        inter = input_flat * targets_flat
        dice_eff = (2 * inter.sum(1) + smooth) / (input_flat.sum(1) + targets_flat.sum(1) + smooth)
        return 1 - dice_eff.sum() / N


class MultiClassDiceLoss(nn.Module):
    def __init__(self, class_weights=None, include_background=False):
        super().__init__()
        self.class_weights = class_weights
        self.include_background = include_background
        self.bdl = BinaryDiceLoss()

    def forward(self, logits, target):
        C = logits.shape[1]
        target_oh = F.one_hot(target.long(), C).permute(0, 3, 1, 2).contiguous()
        probs = F.softmax(logits, dim=1)

        start_c = 0 if self.include_background else 1
        losses = []
        for c in range(start_c, C):
            losses.append(self.bdl(probs[:, c], target_oh[:, c]))

        losses = torch.stack(losses, dim=0)

        if self.class_weights is not None:
            w = self.class_weights.to(losses.device)
            if not self.include_background:
                w = w[1:]
            w = w / (w.mean().clamp_min(1e-12))
            return (losses * w).mean()

        return losses.mean()


# ============================================================
#                 Drive logging + CSV
# ============================================================

class TeeLogger:
    def __init__(self, filename):
        self.terminal = sys.stdout
        self.log = open(filename, "a", encoding="utf8")

    def write(self, message):
        self.terminal.write(message)
        self.log.write(message)

    def flush(self):
        self.terminal.flush()
        self.log.flush()
        try:
            os.fsync(self.log.fileno())
        except Exception:
            pass


def setup_drive_logging(exp_dir):
    logs_dir = os.path.join(exp_dir, "logs")
    os.makedirs(logs_dir, exist_ok=True)
    ts = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    log_path = os.path.join(logs_dir, f"train_{ts}.log")
    sys.stdout = TeeLogger(log_path)
    return log_path


def append_csv(csv_path: str, header: str, line: str):
    exists = os.path.isfile(csv_path)
    with open(csv_path, "a", encoding="utf8") as f:
        if not exists:
            f.write(header)
        f.write(line)
        f.flush()
        try:
            os.fsync(f.fileno())
        except Exception:
            pass


# ============================================================
#                 Option C: estimate weights
# ============================================================

def estimate_ce_weights_from_loader(train_loader, num_classes, device,
                                   max_batches=200, clamp_min=0.1, clamp_max=10.0):
    counts = torch.zeros(num_classes, dtype=torch.float64)
    for k, s in enumerate(train_loader):
        if k >= max_batches:
            break
        y = s["label"].view(-1)
        counts += torch.bincount(y, minlength=num_classes).double()
    freq = counts / counts.sum().clamp_min(1.0)
    w = 1.0 / (freq + 1e-12)
    w = w / w.mean().clamp_min(1e-12)
    w = w.float()
    w = torch.clamp(w, clamp_min, clamp_max).to(device)
    return w, counts


def estimate_dice_weights_from_ce_weights(ce_w):
    w = ce_w.clone().detach().float()
    w[0] = min(float(w[0]), 1.0)
    w = w / w.mean().clamp_min(1e-12)
    return w


# ============================================================
#                 Test wrapper: volume -> 2D slices
# ============================================================

class TestVol2DSliceDataset(Dataset):
    """
    Wrap a test dataset that returns volumes as:
      image: (1, D, H, W) or (D, H, W) or (1, H, W)
      label: same shape
    and expose it as a 2D-slice dataset:
      image: (1, H, W)  (single-channel)
      label: (H, W)
    """
    def __init__(self, base_dataset):
        self.base = base_dataset
        self.index = []  # list of (case_id, slice_id)

        for case_id in range(len(self.base)):
            s = self.base[case_id]
            img = s["image"]
            if isinstance(img, np.ndarray):
                img = torch.from_numpy(img)
            if img.dim() == 4:      # (B,D,H,W) or (C,D,H,W) but in your case (1,D,H,W)
                D = img.shape[1]
            elif img.dim() == 3:    # (D,H,W) or (C,H,W)
                D = img.shape[0]
            else:
                D = 1

            for z in range(D):
                self.index.append((case_id, z))

    def __len__(self):
        return len(self.index)

    def __getitem__(self, idx):
        case_id, z = self.index[idx]
        s = self.base[case_id]

        img = s["image"]
        lab = s["label"]

        if isinstance(img, np.ndarray):
            img = torch.from_numpy(img)
        if isinstance(lab, np.ndarray):
            lab = torch.from_numpy(lab)

        if img.dim() == 4:      # (1,D,H,W)
            img2d = img[0, z]   # (H,W)
        elif img.dim() == 3:    # (D,H,W)
            img2d = img[z]
        elif img.dim() == 2:
            img2d = img
        else:
            raise RuntimeError(f"Unsupported image dim: {img.dim()}")

        if lab.dim() == 4:
            lab2d = lab[0, z]
        elif lab.dim() == 3:
            lab2d = lab[z]
        elif lab.dim() == 2:
            lab2d = lab
        else:
            raise RuntimeError(f"Unsupported label dim: {lab.dim()}")

        img2d = img2d.unsqueeze(0).float()  # (1,H,W)
        lab2d = lab2d.long()                # (H,W)

        return {"image": img2d, "label": lab2d}


# ============================================================
#                 2D prep + TEST eval (strict 2D)
# ============================================================

def _prep_2d_batch(images, labels, img_size, device):
    # images: (B,1,H,W) or (B,3,H,W); labels: (B,H,W)
    if images.size(1) == 1:
        images = images.repeat(1, 3, 1, 1)

    if images.shape[-2] != img_size or images.shape[-1] != img_size:
        images = F.interpolate(images, size=(img_size, img_size), mode="bilinear", align_corners=False)

    if labels.shape[-2] != img_size or labels.shape[-1] != img_size:
        labels = labels.unsqueeze(1).float()
        labels = F.interpolate(labels, size=(img_size, img_size), mode="nearest")
        labels = labels.squeeze(1).long()

    imgs = images.to(device, dtype=torch.float32, non_blocking=True)
    gts = labels.to(device, dtype=torch.long, non_blocking=True)
    return imgs, gts


@torch.no_grad()
def evaluate_on_test_2d(model, test_loader, num_classes, img_size, device):
    model.eval()
    running = init_running_metrics(num_classes)

    for samples in tqdm(test_loader, total=len(test_loader), desc="TEST_2D"):
        images = samples["image"]  # (B,1,H,W)
        labels = samples["label"]  # (B,H,W)

        imgs, gts = _prep_2d_batch(images, labels, img_size, device)
        logits = model(imgs)
        pred = torch.argmax(torch.softmax(logits, dim=1), dim=1)

        pred_np = pred.detach().cpu().numpy()
        gt_np = gts.detach().cpu().numpy()

        for b in range(pred_np.shape[0]):
            running = update_running_metrics(running, pred_np[b], gt_np[b], num_classes)

    per_class, macro = finalize_metrics(running, num_classes)
    return per_class, macro


# ============================================================
#                 Args
# ============================================================

def get_argparser():
    p = argparse.ArgumentParser()

    p.add_argument("--versions_root", type=str, default="/kaggle/input/synapse")
    p.add_argument("--data_subdir", type=str, default="Synapse")
    p.add_argument("--list_dir", type=str, default="/root/lists_synapse")
    p.add_argument("--train_folder", type=str, default="train_npz")
    p.add_argument("--test_folder", type=str, default="test_vol_h5")

    p.add_argument("--num_classes", type=int, default=9)
    p.add_argument("--START_EPOCH", type=int, default=0)
    p.add_argument("--NB_EPOCH", type=int, default=50)

    p.add_argument("--LR", type=float, default=1e-4)
    p.add_argument("--weight_decay", type=float, default=1e-4)

    p.add_argument("--batch_size", type=int, default=4)
    p.add_argument("--test_batch_size", type=int, default=8)
    p.add_argument("--img_size", type=int, default=224)

    p.add_argument("--RESUME", type=bool, default=False)
    p.add_argument("--random_seed", type=int, default=1234)

    p.add_argument("--metrics_every", type=int, default=10)
    p.add_argument("--grad_clip", type=float, default=1.0)

    p.add_argument("--drive_root", type=str, default="/content/drive/MyDrive/Synapse_experiments")
    p.add_argument("--exp_name", type=str, default="Synapse_UNext_only")

    p.add_argument("--ce_weight_batches", type=int, default=200)
    p.add_argument("--ce_w_min", type=float, default=0.1)
    p.add_argument("--ce_w_max", type=float, default=10.0)

    p.add_argument("--save_best_on", type=str, default="macro_dice", choices=["macro_dice", "loss"])

    return p


# ============================================================
#                 Main
# ============================================================

def main():
    opts = get_argparser().parse_known_args()[0]

    drive.mount("/content/drive", force_remount=False)

    exp_dir = os.path.join(opts.drive_root, opts.exp_name)
    os.makedirs(exp_dir, exist_ok=True)
    ckpt_dir = os.path.join(exp_dir, "checkpoints")
    os.makedirs(ckpt_dir, exist_ok=True)

    log_path = setup_drive_logging(exp_dir)
    print("✅ Logging to:", log_path)
    print("✅ Exp dir:", exp_dir)

    csv_path = os.path.join(exp_dir, "history_unext.csv")
    C = opts.num_classes

    header_cols = [
        "epoch", "lr",
        "train_loss", "train_ce", "train_dice_loss",
        "train_macro_dice", "train_macro_jac", "train_macro_hd95",
        "test_macro_dice", "test_macro_jac", "test_macro_hd95",
    ]
    for c in range(1, C):
        header_cols += [
            f"train_dice_c{c:02d}", f"train_jac_c{c:02d}", f"train_hd95_c{c:02d}",
            f"test_dice_c{c:02d}",  f"test_jac_c{c:02d}",  f"test_hd95_c{c:02d}",
        ]
    header = ",".join(header_cols) + "\n"

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print("Device:", device)

    torch.manual_seed(opts.random_seed)
    np.random.seed(opts.random_seed)
    random.seed(opts.random_seed)

    tr_transform = RandomGenerator(output_size=[opts.img_size, opts.img_size])

    train_dataset = Synapse_dataset(
        versions_root=opts.versions_root,
        split="train",
        transform=tr_transform,
        list_dir=opts.list_dir,
        data_subdir=opts.data_subdir,
        train_folder=opts.train_folder,
        test_folder=opts.test_folder,
        verbose=True,
    )
    print("Train size:", len(train_dataset))

    train_loader = DataLoader(
        train_dataset,
        batch_size=opts.batch_size,
        shuffle=True,
        num_workers=2,
        pin_memory=True,
    )

    # Base test dataset (volumes)
    test_base = Synapse_dataset(
        versions_root=opts.versions_root,
        split="test",
        transform=None,
        list_dir=opts.list_dir,
        data_subdir=opts.data_subdir,
        train_folder=opts.train_folder,
        test_folder=opts.test_folder,
        verbose=True,
    )
    print("Test cases:", len(test_base))

    # Wrap volumes -> 2D slices
    test_dataset = TestVol2DSliceDataset(test_base)
    print("Test slices (2D):", len(test_dataset))

    test_loader = DataLoader(
        test_dataset,
        batch_size=opts.test_batch_size,
        shuffle=False,
        num_workers=2,
        pin_memory=True,
    )

    model = UNext(num_classes=C, img_size=opts.img_size).to(device)

    last_path = os.path.join(ckpt_dir, "UNext_last.pth")
    best_path = os.path.join(ckpt_dir, "UNext_best.pth")

    if opts.RESUME and os.path.isfile(last_path):
        print("Loading UNext checkpoint:", last_path)
        model.load_state_dict(torch.load(last_path, map_location=device), strict=True)

    ce_w, counts = estimate_ce_weights_from_loader(
        train_loader, C, device,
        max_batches=opts.ce_weight_batches,
        clamp_min=opts.ce_w_min,
        clamp_max=opts.ce_w_max,
    )
    print("Pixel counts:", counts.cpu().numpy().astype(np.int64))
    print("CE weights:", ce_w.detach().cpu().numpy())

    ce_loss = nn.CrossEntropyLoss(weight=ce_w, reduction="mean")
    dice_w = estimate_dice_weights_from_ce_weights(ce_w)
    dice_loss = MultiClassDiceLoss(class_weights=dice_w, include_background=False)

    optimizer = torch.optim.AdamW(model.parameters(), lr=opts.LR, weight_decay=opts.weight_decay)

    best_score = -1e9 if opts.save_best_on == "macro_dice" else 1e9

    def _fmt(x):
        return "" if (x is None or (isinstance(x, float) and np.isnan(x))) else f"{x}"

    for epoch in range(opts.START_EPOCH, opts.NB_EPOCH):
        lr = optimizer.param_groups[0]["lr"]

        print("\n" + "=" * 70)
        print(f"Epoch {epoch}/{opts.NB_EPOCH-1} | lr={lr:.6g} | FIXED LR (2D test)")
        print("=" * 70)

        model.train()
        running_train = init_running_metrics(C)

        loss_sum = 0.0
        ce_sum = 0.0
        dice_sum = 0.0
        n_batches = 0

        for i, samples in tqdm(enumerate(train_loader), total=len(train_loader), desc="TRAIN"):
            images = samples["image"]
            labels = samples["label"]

            if images.size(1) == 1:
                images = images.repeat(1, 3, 1, 1)

            imgs = images.to(device, dtype=torch.float32, non_blocking=True)
            gts = labels.to(device, dtype=torch.long, non_blocking=True)

            optimizer.zero_grad(set_to_none=True)
            logits = model(imgs)

            loss_ce = ce_loss(logits, gts)
            loss_d = dice_loss(logits, gts)
            loss = loss_ce + loss_d

            loss.backward()
            if opts.grad_clip > 0:
                torch.nn.utils.clip_grad_norm_(model.parameters(), opts.grad_clip)
            optimizer.step()

            loss_sum += float(loss.detach().cpu())
            ce_sum += float(loss_ce.detach().cpu())
            dice_sum += float(loss_d.detach().cpu())
            n_batches += 1

            if i % 20 == 0:
                print(f"[batch {i:04d}] loss={loss.item():.4f} | ce={loss_ce.item():.4f} | dice={loss_d.item():.4f}")

            if opts.metrics_every > 0 and (i % opts.metrics_every == 0):
                with torch.no_grad():
                    pred = torch.argmax(torch.softmax(logits, dim=1), dim=1)
                    pred_np = pred.detach().cpu().numpy()
                    gt_np = gts.detach().cpu().numpy()
                    for b in range(pred_np.shape[0]):
                        running_train = update_running_metrics(running_train, pred_np[b], gt_np[b], C)

        train_loss = loss_sum / max(1, n_batches)
        train_ce = ce_sum / max(1, n_batches)
        train_dice_l = dice_sum / max(1, n_batches)

        train_per_class, train_macro = finalize_metrics(running_train, C)
        tr_macro_dice, tr_macro_jac, tr_macro_hd95 = train_macro

        print("\n--- Train Summary ---")
        print(f"Loss={train_loss:.6f} | CE={train_ce:.6f} | DiceLoss={train_dice_l:.6f}")
        print(f"[TRAIN] Macro Dice={tr_macro_dice:.4f} | Jaccard={tr_macro_jac:.4f} | HD95={tr_macro_hd95:.4f}")

        test_per_class, test_macro = evaluate_on_test_2d(model, test_loader, C, opts.img_size, device)
        te_macro_dice, te_macro_jac, te_macro_hd95 = test_macro

        print("\n--- Test Summary (2D slices from test_vol) ---")
        print(f"[TEST] Macro Dice={te_macro_dice:.4f} | Jaccard={te_macro_jac:.4f} | HD95={te_macro_hd95:.4f}")

        torch.save(model.state_dict(), last_path)
        print("✅ Saved UNext last checkpoint:", last_path)

        row = [
            str(epoch), f"{lr}",
            f"{train_loss}", f"{train_ce}", f"{train_dice_l}",
            _fmt(tr_macro_dice), _fmt(tr_macro_jac), _fmt(tr_macro_hd95),
            _fmt(te_macro_dice), _fmt(te_macro_jac), _fmt(te_macro_hd95),
        ]
        for c in range(1, C):
            td, tj, th = train_per_class[c]
            vd, vj, vh = test_per_class[c]
            row += [_fmt(td), _fmt(tj), _fmt(th), _fmt(vd), _fmt(vj), _fmt(vh)]
        append_csv(csv_path, header, ",".join(row) + "\n")

        if opts.save_best_on == "macro_dice":
            score = te_macro_dice
            if (not np.isnan(score)) and score > best_score:
                best_score = score
                torch.save(model.state_dict(), best_path)
                print(f"✅ Saved UNext best checkpoint (test_macro_dice={best_score:.4f}):", best_path)
        else:
            score = train_loss
            if score < best_score:
                best_score = score
                torch.save(model.state_dict(), best_path)
                print(f"✅ Saved UNext best checkpoint (loss={best_score:.6f}):", best_path)

    torch.cuda.empty_cache()
    print("Done.")


if __name__ == "__main__":
    main()

TEST_2D:  60%|██████    | 118/196 [05:49<04:11,  3.22s/it]